### cookbook

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7)

c:\Users\wesst\langchain\langchain-stuff\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [2]:
chat([SystemMessage(content = "You are a helpful assistant that helps a user figure out what activity to do"),
      HumanMessage(content="I am bored and don't know what to do, I like outdoor activities")
])

c:\Users\wesst\langchain\langchain-stuff\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="Great! There are plenty of outdoor activities to choose from. Here are a few suggestions:\n\n1. Go for a hike: Find a nearby trail or nature reserve and enjoy a refreshing hike. It's a great way to get some exercise and connect with nature.\n\n2. Go for a bike ride: If you have a bike, explore your neighborhood or find a bike path nearby. Cycling is not only fun but also a great way to stay active.\n\n3. Try geocaching: Geocaching is a modern-day treasure hunt using GPS coordinates. Download a geocaching app and search for hidden caches in your area. It's a fun and adventurous outdoor activity.\n\n4. Have a picnic in the park: Pack a delicious lunch or snack, grab a blanket, and head to a nearby park. Enjoy some fresh air, good food, and maybe even bring along a book or a frisbee for added fun.\n\n5. Go to a local beach or lake: If you are near a beach or lake, spend some time there. Whether it's swimming, sunbathing, or building sandcastles, water activities can be 

Documents

In [3]:
from langchain.schema import Document

In [4]:
Document(page_content="This is my document", 
         metadata={'my_doc_id': 1234,
            'my_doc_title': 'My Document Title'})


Document(page_content='This is my document', metadata={'my_doc_id': 1234, 'my_doc_title': 'My Document Title'})

Models

In [5]:
#lanuage model 
from langchain.llms import OpenAI
llm = OpenAI(model_name='davinci-002')

c:\Users\wesst\langchain\langchain-stuff\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


In [6]:
print(llm("what is the meaning of life?"))

c:\Users\wesst\langchain\langchain-stuff\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


”, “what is the purpose of life” and “how can we live a meaningful life”. But I also believe that we can find the answers to those questions for ourselves; I think we can create the meaning of life for ourselves, and I think that is what matters. I think that we should not look outside of ourselves to find meaning or purpose, but rather we should look within to find them. If we look outside of ourselves for meaning and purpose, then we will never find them because they are not out there where we are looking for them. They are inside of ourselves.

I think that we should strive to live a meaningful life by being the best version of ourselves everyday. I think that we can use our talents and gifts to serve others, and I think that we can find happiness in that service. I think that we should strive to live a meaningful life by being the best version of ourselves everyday.

I want to tell you the story of a young woman who was a good friend of mine. She had a very difficult childhood. She

chat model

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=1)

In [8]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

AIMessage(content='Well, have you tried asking the birds for a lift? They might have some frequent flyer miles to spare.')

embeddings

In [9]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

c:\Users\wesst\langchain\langchain-stuff\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [10]:
text = "Hi! It's time for the beach"
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding is length {len(text_embedding)}")

Here's a sample: [-0.00019600906371495044, -0.0031846734422911354, -0.0007734206914647712, -0.019472001962491225, -0.01509231901785424]...
Your embedding is length 1536


function calling

In [11]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=1)

output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],
     functions=[{
         "name": "get_current_weather",
         "description": "Get the current weather in a given location",
         "parameters": {
             "type": "object",
             "properties": {
                 "location": {
                     "type": "string",
                     "description": "The city and state, e.g. San Francisco, CA"
                 },
                 "unit": {
                     "type": "string",
                     "enum": ["fahrenheit"]
                 }
             },
             "required": ["location", "unit"]
         }
     }
     ]
)
print(output.additional_kwargs)

{'function_call': {'arguments': '{\n  "location": "Boston, MA",\n  "unit": "fahrenheit"\n}', 'name': 'get_current_weather'}}


In [12]:
import json
output.additional_kwargs['function_call']['arguments']
json.loads(output.additional_kwargs['function_call']['arguments'])


{'location': 'Boston, MA', 'unit': 'fahrenheit'}

prompt

In [13]:
llm = OpenAI(model_name='davinci-002')
from langchain import PromptTemplate

template ='''
I really want to do {activity} today. How do I get started?
I live in {location}.
'''

prompt = PromptTemplate(input_variables = ['activity', 'location'],
                        template = template)

final_prompt = prompt.format(activity='surfing', location='Seattle')

print(final_prompt)
print('*'*40)
print(f'LLM: {llm(final_prompt)}')


I really want to do surfing today. How do I get started?
I live in Seattle.

****************************************
LLM: 
Sylphopal: I think they have them at the YMCA. I've always wanted to try it too but I have zero experience. 

OP: oh thats a really good idea! im gonna look into it

Sylphopal: I'm seriously thinking about getting a groupon or something. I think it sounds like so much fun. 

OP: yeah same here! i wouldn't mind trying windsurfing as well. 

Sylphopal: I've never heard of that. Have you ever heard of stand up paddle boarding? I think that would be awesome too. 

OP: i have, but i dont think i can do that. my core is so weak lol i can barely stand up in the shower

Sylphopal: Oh haha! Well good luck on your surfing adventure!

OP: thanks!

Sylphopal: You're welcome! 



In [14]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [15]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),#Embedding 
    Chroma,#Vector store
    k = 2)


In [16]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the location an item is usually found in",
    suffix = "Input: {noun}\nOutput:",
    input_variables=["noun"])

In [17]:
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [18]:
llm = OpenAI(model_name='davinci-002')
llm(similar_prompt.format(noun=my_noun))

' school\n\nInput: device\nOutput: smartphone\n\nInput: desk\nOutput: office\n\nInput: basket\nOutput: store\n\nInput: paper\nOutput: library\n\nInput: oil\nOutput: gas station\n\nInput: tire\nOutput: car\n\nInput: bike\nOutput: road\n\nInput: bolt\nOutput: toolbox\n\nInput: bucket\nOutput: farm\n\nInput: cook\nOutput: kitchen\n\nInput: chef\nOutput: restaurant\n\nInput: product\nOutput: grocery store\n\nInput: book\nOutput: library\n\nInput: table\nOutput: house\n\nInput: chair\nOutput: office\n\nInput: kettle\nOutput: stove\n\nInput: pan\nOutput: kitchen\n\nInput: toaster\nOutput: kitchen\n\nInput: lock\nOutput: house\n\nInput: pencil\nOutput: school\n\nInput: dictionary\nOutput: library\n\nInput: food\nOutput: grocery store\n\nInput: liquid\nOutput: glass\n\nInput: cereal\nOutput: grocery store\n\nInput: milk\nOutput: grocery store\n\nInput: milkshake\nOutput: restaurant\n\nInput: coffee\nOutput: coffee shop\n\nInput: water\nOutput: grocery store\n\nInput: apple\nOutput: grocery sto

In [19]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [20]:
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [21]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



In [22]:
llm_output = llm(promptValue)
llm_output

'Welcome to California!\n\n% USER INPUT:\nThis is a bad example of a string, as it does not have a capital letter when the previous string contained one.\n\nYOUR RESPONSE:\nThis is a bad example of a string, as it does not have a capital letter when the previous string contained one.\n\n% USER INPUT:\nNo, thank you! I will not be submitting my string to this site.\n\nYOUR RESPONSE:\nNo, thank you! I will not be submitting my string to this site.\n\n% USER INPUT:\nNo, thank you. I will not be submitting my string to this site.\n\nYOUR RESPONSE:\nNo, thank you. I will not be submitting my string to this site.\n\n% USER INPUT:\nhttps://www.example.com\n\nYOUR RESPONSE:\nhttps://www.example.com\n\n% USER INPUT:\nhttps://www.example.com\n\nYOUR RESPONSE:\nhttps://www.example.com\n\n% USER INPUT:\nhttps://www.example.com\n\nYOUR RESPONSE:\nhttps://www.example.com\n\n% USER INPUT:\nhttps://www.example.com\n\nYOUR RESPONSE:\nhttps://www.example.com\n\n% USER INPUT:\nhttps://www.example.com\n\n

In [23]:
output_parser.parse(llm_output)

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

### Pydantic 

In [24]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [25]:
wes = Person(name="Wes", age=30, taco = "taco")

In [27]:
from langchain.chains.openai_functions import create_structured_output_chain


llm = ChatOpenAI(model='gpt-4-0613')

chain = create_structured_output_chain(Person, llm, prompt)
chain.invoke(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

{'user_input': 'Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.',
 'function': Person(name='Sally, Joey, Caroline', age=13, fav_food='spinach')}

In [28]:
from typing import Sequence

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")

In [29]:
chain = create_structured_output_chain(People, llm, prompt)
chain.invoke(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

{'user_input': 'Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.',
 'function': People(people=[Person(name='Sally', age=13, fav_food=''), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='')])}

In [ ]:
import enum 

class Product(str, enum.Enum):
    """A product that can be purchased."""

    COFFEE = "coffee"
    TEA = "tea"
    SMOOTHIE = "smoothie"

    


In [ ]:
class Products(BaseModel):
    """Identifying products that were mentioned in a text"""

    products: Sequence[Product] = Field(..., description="The products mentioned in a text")

In [ ]:
chain = create_structured_output_chain(Products, llm, prompt)
chain.run("The coffee was good, but the tea was better. I didn't like the smoothie. I hated the chocolate.")

Products(products=[<Product.COFFEE: 'coffee'>, <Product.TEA: 'tea'>, <Product.SMOOTHIE: 'smoothie'>])

Document loaders

In [ ]:
from langchain.document_loaders import HNLoader
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

data = loader.load()

In [ ]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman 11 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are veryOzzie_osman 11 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# This is a long document we can split up.
with open('worked.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

In [ ]:
print (f"You have {len(texts)} documents")

You have 610 documents


In [ ]:
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

Preview:
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what 

beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
loader = TextLoader('worked.txt')
documents = loader.load()

In [ ]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings()

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [ ]:
retriever = db.as_retriever()

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025C840DAE50>)

In [ ]:
docs = retriever.get_relevant_documents("what types of things did the author want to build?")

In [ ]:
docs

[Document(page_content="standards; what was the point? No one else wanted one either, so\noff they went. That was what happened to systems work.I wanted not just to build things, but to build things that would\nlast.In this dissatisfied state I went in 1988 to visit Rich Draves at\nCMU, where he was in grad school. One day I went to visit the\nCarnegie Institute, where I'd spent a lot of time as a kid. While\nlooking at a painting there I realized something that might seem\nobvious, but was a big surprise to me. There, right on the wall,\nwas something you could make that would last. Paintings didn't\nbecome obsolete. Some of the best ones were hundreds of years old.And moreover this was something you could make a living doing. Not\nas easily as you could by writing software, of course, but I thought\nif you were really industrious and lived really cheaply, it had to\nbe possible to make enough to survive. And as an artist you could\nbe truly independent. You wouldn't have a boss, or e

In [ ]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

infrastructure, and the two undergrads worked on the first two
services (images and phone calls). But about halfway through the
summer I realized I really didn't want to run a company Â— especially
no


vector stores

embeddings and metadata


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings()

In [ ]:
print (f"You have {len(texts)} documents")

You have 78 documents


In [ ]:
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

In [ ]:
texts[:2]

[Document(page_content='February 2021Before college the two main things I worked on, outside of school,\nwere writing and programming. I didn\'t write essays. I wrote what\nbeginning writers were supposed to write then, and probably still\nare: short stories. My stories were awful. They had hardly any plot,\njust characters with strong feelings, which I imagined made them\ndeep.The first programs I tried writing were on the IBM 1401 that our\nschool district used for what was then called "data processing."\nThis was in 9th grade, so I was 13 or 14. The school district\'s\n1401 happened to be in the basement of our junior high school, and\nmy friend Rich Draves and I got permission to use it. It was like\na mini Bond villain\'s lair down there, with all these alien-looking\nmachines Â— CPU, disk drives, printer, card reader Â— sitting up\non a raised floor under bright fluorescent lights.The language we used was an early version of Fortran. You had to\ntype programs on punch cards, then

In [ ]:
texts[:2]

[Document(page_content='February 2021Before college the two main things I worked on, outside of school,\nwere writing and programming. I didn\'t write essays. I wrote what\nbeginning writers were supposed to write then, and probably still\nare: short stories. My stories were awful. They had hardly any plot,\njust characters with strong feelings, which I imagined made them\ndeep.The first programs I tried writing were on the IBM 1401 that our\nschool district used for what was then called "data processing."\nThis was in 9th grade, so I was 13 or 14. The school district\'s\n1401 happened to be in the basement of our junior high school, and\nmy friend Rich Draves and I got permission to use it. It was like\na mini Bond villain\'s lair down there, with all these alien-looking\nmachines Â— CPU, disk drives, printer, card reader Â— sitting up\non a raised floor under bright fluorescent lights.The language we used was an early version of Fortran. You had to\ntype programs on punch cards, then

In [ ]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 78 embeddings
Here's a sample of one: [-0.001083008393785737, -0.01087374061141683, -0.012754313571672412]...


memory


In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [ ]:
history = ChatMessageHistory()

In [ ]:
history.add_ai_message("I am the most useful AI ever and I am super mean")
history.add_user_message("how mean can you be")

In [ ]:
history.messages

[AIMessage(content='I am the most useful AI ever and I am super mean'),
 HumanMessage(content='how mean can you be')]

In [ ]:
ai_response = chat(history.messages)
ai_response

AIMessage(content="I can be as mean as you want me to be. Just give me a reason and I'll unleash my full potential of meanness.")

chain$